<a href="https://colab.research.google.com/github/aayushbhurtel/Data_Visualization/blob/main/Quiz5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Visualization
## Aayush Bhurtel

Here I am authenticating google bigQueries API with my account.

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Firstly, I am going to print 5 rows of data from our dataset to remind us what is in the dataset and how can we plan to perform actions according to our assignment.

In [ ]:
%%bigquery --project data-viz-big-query
SELECT *
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
LIMIT 5

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01+00:00,1006,Zilker Park West,1008,Nueces @ 3rd,39
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00+00:00,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19


#What was the second most popular starting station?

here I am using professor starting code to display number of trips and start station name side by side to compare where I can see the most popular starting station.

In [58]:


%%bigquery --project data-viz-big-query

SELECT COUNT(*) as number_trips, start_station_name
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
ORDER BY number_trips DESC



,number_trips,start_station_name
0,72799,21st & Speedway @PCL
1,40635,Riverside @ S. Lamar
2,36520,City Hall / Lavaca & 2nd
3,35307,2nd & Congress
4,34758,Rainey St @ Cummings
...,...,...
188,4,Marketing Event
189,2,Eeyore's 2018
190,1,Stolen
191,1,Eeyore's 2017


Now, I will be creating a subquery to calculate the second most popular starting station. 

In [59]:
%%bigquery --project data-viz-big-query
SELECT start_station_name,
 COUNT(*) as most_pop_trip
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name

HAVING most_pop_trip!= (SELECT MAX(most_pop_trip)as max_trip
FROM (
SELECT COUNT(*) as most_pop_trip, start_station_name
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
))
ORDER BY most_pop_trip DESC
LIMIT 1

,start_station_name,most_pop_trip
0,Riverside @ S. Lamar,40635


using professor starting code to calculate the trips that lasted over an hour and were a round trip.

In [61]:


%%bigquery --project data-viz-big-query

SELECT start_station_name, duration_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name = end_station_name
LIMIT 100



,start_station_name,duration_minutes
0,Toomey Rd @ South Lamar,31
1,Toomey Rd @ South Lamar,31
2,Toomey Rd @ South Lamar,30
3,State Capitol @ 14th & Colorado,19
4,State Capitol @ 14th & Colorado,17
...,...,...
95,Rainey @ River St,87
96,Zilker Park West,11
97,ACC - West & 12th Street,17
98,Rainey @ River St,87


Here I calculate the number of trips took over an hour.

In [60]:
%%bigquery --project data-viz-big-query

SELECT *, COUNTIF(duration_minutes>60) as number_of_trip_took_over_an_hour
FROM (
  SELECT start_station_name, duration_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name = end_station_name
)
GROUP BY duration_minutes, start_station_name
Order BY number_of_trip_took_over_an_hour DESC

,start_station_name,duration_minutes,number_of_trip_took_over_an_hour
0,Riverside @ S. Lamar,62,109
1,Riverside @ S. Lamar,61,95
2,Riverside @ S. Lamar,63,94
3,Riverside @ S. Lamar,69,90
4,Zilker Park,61,88
...,...,...,...
24822,26th/Nueces,40,0
24823,26th/Nueces,17,0
24824,26th/Nueces,53,0
24825,26th/Nueces,48,0
